In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Preprocessing Criterias

1. age between 18 and 90 ☑️  
2. hospitalized for at least 2 days ☑️  
3. (abandoned)at least 2 SCr records between Admit date and AKI onset date, and also at least 2 SCr records within 30 days before the current Admit
date, thus, one patient should at least have 5 records, 2 from history, 2 between admit and onset and 1 from the AKI onset day. ☑️  
4. Exclude:  
    (1)Non-AKI patients, Patients directly starting with AKI stage 2 or 3 ☑️  
    (2)eGFR <15 mL/min/1.73 m^2 (calculated) ☑️    
    (3)has undergone any dialysis procedure or renal transplantation (RRT) prior to the visit (find it in PX) ☑️  
    (4)required RRT within 48h of their admission SCr measurement record ☑️    
    (5)has pre-existing end stage renal disease ☑️      
    (6)Burn patients ☑️     
    (7)initial SCr measurement of > 3.5 mg/dL ☑️    
    
General Idea: Use AKI_ONSET to filter out NON-AKI patients, and hospitalized for less than 2 days, initial SCr measurement of > 3.5 mg/dL, then use it as query, to screen AKI_DEMO(for age), AKI_PX(for dialysis and RRT), AKI_DX(for end stage renal disease, burn patients)

In [ ]:
#tunable parameters'
time_interval_before_ONSET = 7
min_hospital_time = 3
missing_percent_allowed = 0.2
%store time_interval_before_ONSET
n_rec_after_admit = 3
max_SCR = 4

In [ ]:
#define raw path
raw_path = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/'
%store raw_path
#pandas can show all the columns
pd.set_option('display.max_columns', None)
#all avaiable clinical center names
ct_names = ['UPITT', 'UTHSCSA', 'UIOWA', 'UTSW', 'MCW', 'UMHC', 'UofU', 'KUMC_ORCALE', 'UNMC']
%store ct_names 

# Modification (Oct 6, 2023)
1. extract scr records 3 days prior to discharge and get the mean, then we can calculate % of patients went below to baseline
2. persistent AKI definition: AKI onset lasts for more than 48h from its onset, get the % of persistent AKI in each cluster

# What Data Should Be Collected

According to Ho Yin's paper, we need to collect AKI stages, Age, Gender, Race, BMI, ALT, AST, AMMONIA, CALCIUM, CK, CK_MB

# Create a df that records the number

In [ ]:
records_num_df = pd.DataFrame(0, index = ['Total number of encounters', 'Patients experiencing AKI onset during hospitalization',
                                         'Patients whose first onset AKI stage was stage 1', 'Hospitalization > 2 days', 
                                         'SCr baseline less than 3.5', 'Patients with eGFR < 15 excluded', 'Burned patients excluded', 
                                         'Patients with pre-existing end stage renal disease excluded', 
                                          'Patients with dialysis or renal transplantation excluded',
                                         'Patient not satisfying SCr trajectory requirements excluded'], columns = ct_names)

# Preprocess ONSETS Data and Screen AKI Patients

In [ ]:
#need to uppercase all the column names of MCW
def Upper_Case_Columns(df):
    df.columns = [col.upper() for col in df.columns] 

In [ ]:
def read_AKI_ONSETS(ct_names, raw_path, fill_in_rec_num_df = False):
    
    AKI_ONSETS_dfs = dict()
    
    for ct in ct_names:
        print('\n' + ct + ':')
        data_path = raw_path + ct + '/raw/'
        if (ct == 'UPITT') or (ct == 'UTHSCSA') or (ct == 'UIOWA') or (ct == 'UNMC'):
            AKI_onset = pd.read_csv(data_path + "AKI_ONSETS.csv", delimiter = ',')
        elif (ct == 'UTSW'):
            AKI_onset = pd.read_csv(data_path + "AKI_ONSETS.dsv", delimiter = '|')
        elif (ct == 'MCW'):
            AKI_onset = pd.read_csv(data_path + "AKI_ONSETS.dsv", delimiter = '|')
            Upper_Case_Columns(AKI_onset)
        elif (ct == 'UMHC'):
            AKI_onset = pd.read_csv(data_path + "DEID_AKI_ONSETS.csv", delimiter = ',')
        elif (ct == 'UofU'):
            AKI_onset = pd.read_csv(data_path + "AKI_ONSETS.csv", delimiter = '|')
        elif (ct == 'KUMC'):
            AKI_onset = pd.read_csv(data_path + "AKI_ONSETS.csv", delimiter = ',')
            AKI_onset_cols = AKI_onset.columns.tolist()
            # AKI_onset_cols = [s[:-len('"+PD.DATE_SHIFT"')] \
            #                   if s.endswith('"+PD.DATE_SHIFT"') else s for s in AKI_onset_cols]
            AKI_onset.columns = AKI_onset_cols
        
        AKI_onset.rename(columns={'ENCOUNTERID': 'ONSETS_ENCOUNTERID'}, inplace = True) 
        
        AKI_ONSETS_dfs[ct] = AKI_onset
        print('Initially, there are %d encounters in total!' %(len(AKI_ONSETS_dfs[ct].ONSETS_ENCOUNTERID.unique())))
        
        if fill_in_rec_num_df:
            records_num_df.loc['Total number of encounters', ct] = len(AKI_ONSETS_dfs[ct].ONSETS_ENCOUNTERID.unique())
            
    return AKI_ONSETS_dfs

In [ ]:
def read_AKI_ONSETS_v2(ct_names, fill_in_rec_num_df = False):
    new_onsets = pd.read_csv('/blue/yonghui.wu/lideyi/AKI_subphenotyping_project_v3/NEW_ONSETS/NEW_ONSETS.csv')
    AKI_ONSETS_dfs = dict()
    for ct in ct_names:
        print('\n' + ct + ':')
        if ct == 'KUMC_ORCALE':
            this_center = new_onsets[new_onsets.CENTER_NAME == 'KUMC'].copy()
        else:
            this_center = new_onsets[new_onsets.CENTER_NAME == ct].copy()
        
        AKI_ONSETS_dfs[ct] = this_center
        print('Initially, there are %d encounters in total!' %(len(AKI_ONSETS_dfs[ct].ONSETS_ENCOUNTERID.unique())))
        if fill_in_rec_num_df:
            records_num_df.loc['Total number of encounters', ct] = len(AKI_ONSETS_dfs[ct].ONSETS_ENCOUNTERID.unique())
    return AKI_ONSETS_dfs

In [ ]:
def generate_query_from_ONSETS(AKI_ONSETS_dfs):
    
    AKI_query_dfs = dict()
    
    for ct, AKI_onset in AKI_ONSETS_dfs.items():
        print('\n' + ct + ':')
        #Criteria 4 exclude (1) screen Non-AKI patients
        AKI_onset = AKI_onset[AKI_onset.NONAKI_SINCE_ADMIT == False]
        print('After Criteria 4 Exclude (1) screen Non-AKI patients, there are %d qualified encounters left!' 
              %(len(AKI_onset.ONSETS_ENCOUNTERID.unique())))
        records_num_df.loc['Patients experiencing AKI onset during hospitalization', ct] = \
        len(AKI_onset.ONSETS_ENCOUNTERID.unique())
        
        #only extract useful columns
        AKI_onset = AKI_onset[['PATID','ONSETS_ENCOUNTERID','ADMIT_DATE', 'DISCHARGE_DATE', 
                               'SERUM_CREAT_BASE', 'AKI1_ONSET', 'AKI2_ONSET', 'AKI3_ONSET']]
        
        # Converting string data type into datetime object
        time_cols = ['ADMIT_DATE', 'DISCHARGE_DATE', 'AKI1_ONSET', 'AKI2_ONSET', 'AKI3_ONSET']
        for col in time_cols:
            AKI_onset[col] = pd.to_datetime(AKI_onset[col], format='mixed')

        #filter patients not start with AKI stage 1
        AKI_onset = AKI_onset[AKI_onset.apply(filter_patients_start_stage_1, axis=1)]
        print('After Criteria 4 Exclude (1.1) screen Non-AKI Stage 1 patients, there are %d qualified encounters left!' 
              %(len(AKI_onset.ONSETS_ENCOUNTERID.unique())))
        records_num_df.loc['Patients whose first onset AKI stage was stage 1', ct] = \
        len(AKI_onset.ONSETS_ENCOUNTERID.unique())
        
        #find highest stage    
        AKI_onset['HIGHEST_STAGE'] = AKI_onset.apply(find_highest_stage, axis = 1)
        AKI_onset['HIGHEST_STAGE_ONSET'] = AKI_onset.apply(find_highest_stage_onset_date, axis = 1)
        AKI_onset['PROGRESSION_TIME'] = AKI_onset.apply(progression_time, axis = 1)

        #Criteria 2: hospitalized for at least 3 days
        AKI_onset = AKI_onset[(AKI_onset.DISCHARGE_DATE - AKI_onset.ADMIT_DATE).dt.days >= min_hospital_time]
        print('After Criteria 2: hospitalized for at least %s days, there are %d qualified encounters left!' 
              %(min_hospital_time, len(AKI_onset.ONSETS_ENCOUNTERID.unique())))
        records_num_df.loc['Hospitalization > 2 days', ct] = \
        len(AKI_onset.ONSETS_ENCOUNTERID.unique())

        #Criteria 4 Exclude (7): initial SCr measurement of > 3.5 mg/dL
        AKI_onset = AKI_onset[AKI_onset.SERUM_CREAT_BASE <= 3.5]
        print('After Criteria 4 Exclude (7): initial SCr measurement of > 3.5 mg/dL, there are %d qualified encounters left!' 
              %(len(AKI_onset.ONSETS_ENCOUNTERID.unique())))
        records_num_df.loc['SCr baseline less than 3.5', ct] = \
        len(AKI_onset.ONSETS_ENCOUNTERID.unique())

        #drop any possible duplicates
        AKI_onset.drop_duplicates(inplace = True)

        #make a deep copy for further preprocessing with other dataframes
        AKI_query = AKI_onset.copy(deep=True)
        AKI_query.rename(columns={'AKI1_ONSET': 'EARLIEST_ONSET'}, inplace=True)
        AKI_query.drop(['AKI2_ONSET', 'AKI3_ONSET'], axis = 1, inplace = True)
        
        
        AKI_query_dfs[ct] = AKI_query
        
    return AKI_query_dfs

In [ ]:
def progression_time(row):
    return (row['HIGHEST_STAGE_ONSET'] - row['AKI1_ONSET']).days

In [ ]:
def find_highest_stage_onset_date(row):
    if row['HIGHEST_STAGE'] == 'AKI Stage 3':
        return row['AKI3_ONSET']
    elif row['HIGHEST_STAGE'] == 'AKI Stage 2':
        return row['AKI2_ONSET']
    else:
        return row['AKI1_ONSET']

In [ ]:
#some patients have both 2 stages or 3 stages, find the highest one
def find_highest_stage(row):
    if pd.notna(row['AKI3_ONSET']):
        return 'AKI Stage 3'
    elif pd.notna(row['AKI2_ONSET']):
        return 'AKI Stage 2'
    else:
        return 'AKI Stage 1'

In [ ]:
def filter_patients_start_stage_1(row):
    if pd.notna(row['AKI1_ONSET']):
        #patients only developed stage 1
        if (pd.isna(row['AKI2_ONSET'])) and (pd.isna(row['AKI3_ONSET'])):
            return True
        #patients developed stage 1 at first
        elif row['AKI1_ONSET'] < min(row[['AKI2_ONSET', 'AKI3_ONSET']]):
            return True
    return False

In [ ]:
#read onset datasets, which indicates which patient experienced AKI during hosipitalization
AKI_ONSETS_dfs = read_AKI_ONSETS_v2(ct_names, True)

In [ ]:
AKI_query_dfs = generate_query_from_ONSETS(AKI_ONSETS_dfs)

# Preprocess Demographic Data and Join with Query 

In [ ]:
#read patients' demographical data
def read_AKI_DEMO(ct_names, raw_path):
    AKI_DEMO_dfs = dict()
    use_cols = ['ONSETS_ENCOUNTERID', 'AGE', 'PATID', 'SEX', 'RACE']

    for ct in ct_names:
        data_path = raw_path + ct + '/raw/'
        if (ct == 'UPITT') or (ct == 'UTHSCSA') or (ct == 'UIOWA') or (ct == 'KUMC_ORCALE'):
             AKI_DEMO = pd.read_csv(data_path + "AKI_DEMO.csv", delimiter = ',', usecols = use_cols)
        elif (ct == 'UTSW'):
            AKI_DEMO = pd.read_csv(data_path + "AKI_DEMO.dsv", delimiter = '|', usecols = use_cols)
        elif (ct == 'MCW'):
            AKI_DEMO = pd.read_csv(data_path + "AKI_DEMO.dsv", delimiter = '|', usecols = list(map(str.lower, use_cols)))
            Upper_Case_Columns(AKI_DEMO)
        elif (ct == 'UMHC'):
            AKI_DEMO = pd.read_csv(data_path + "DEID_AKI_DEMO.csv", delimiter = ',', usecols = use_cols)
        elif (ct == 'UofU'):
            AKI_DEMO = pd.read_csv(data_path + "AKI_DEMO.csv", delimiter = '|', 
                                           header=None, skiprows = 1, usecols=[0, 1, 2, 5, 17])
            AKI_DEMO.columns = use_cols
    
        AKI_DEMO_dfs[ct] = AKI_DEMO
        
    return AKI_DEMO_dfs

In [ ]:
#calculate eGFR
def calculate_ckd_epi(row, SCr_col_name):
    
    age = row['AGE']
    gender = row['SEX']
    race = row['RACE']
    SCr = row[SCr_col_name]
    
    # Constants for the CKD-EPI formula
    k = 0.7 if gender == 'F' else 0.9
    alpha = -0.329 if gender == 'F' else -0.411
    
    # Calculate the eGFR
    min_term = min(SCr / k, 1) ** alpha
    max_term = max(SCr / k, 1) ** -1.209
    age_term = 0.993 ** age
    # Gender and ethnicity adjustments
    gender_term = 1.018 if gender == 'F' else 1
    african_american_term = 1.159 if race == "RACE:black" else 1
    
    eGFR = 141 * min_term * max_term * age_term * gender_term * african_american_term
    
    return eGFR

In [ ]:
def preprocess_query_by_AKI_DEMO(AKI_DEMO_dfs, AKI_query_dfs):
    
    for ct, AKI_DEMO in AKI_DEMO_dfs.items():
        print('\n' + ct + ':')
        AKI_DEMO = AKI_query_dfs[ct].merge(AKI_DEMO, how = 'left', on = ['PATID', 'ONSETS_ENCOUNTERID'])
        
        #Criteria 1: age between 18 and 90
        AKI_DEMO = AKI_DEMO[(AKI_DEMO.AGE >= 18) & (AKI_DEMO.AGE <= 90)]
        AKI_query_dfs[ct] = AKI_query_dfs[ct][AKI_query_dfs[ct].PATID.isin(AKI_DEMO.PATID)]
        print('After Criteria 1: age between 18 and 90, there are %d qualified patients left!' 
              %(len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())))
        
        #Criteria 4 Exclude (2): eGFR <15 mL/min/1.73 m^2 
        AKI_DEMO['GFR'] = AKI_DEMO.apply(calculate_ckd_epi, args = ('SERUM_CREAT_BASE',), axis = 1)
        AKI_DEMO = AKI_DEMO[AKI_DEMO.GFR >= 15]

        #update query by filtering out AKI patients that is not with the age range
        AKI_query_dfs[ct] = AKI_query_dfs[ct][AKI_query_dfs[ct].PATID.isin(AKI_DEMO.PATID)]
        print('After Criteria 1: Exclude(2) GFR < 15, there are %d qualified patients left!' 
              %(len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())))
        records_num_df.loc['Patients with eGFR < 15 excluded', ct] = \
        len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())

In [ ]:
AKI_DEMO_dfs = read_AKI_DEMO(ct_names, raw_path)

In [ ]:
preprocess_query_by_AKI_DEMO(AKI_DEMO_dfs, AKI_query_dfs)

# Preprocess Diagnosis Data and Join with Query

In [ ]:
#read patients' diagnosis data
#cneters do not have a DX_DATE: UTHSCSA, UTSW, UofU
def read_AKI_DX(ct_names, raw_path):
    AKI_DX_dfs = dict()
    use_cols = ['PATID', 'DX_DATE', 'DX', 'DX_TYPE', 'DAYS_SINCE_ADMIT']
    ct_missing_DX_DATE = ['UTHSCSA', 'UTSW', 'UofU']
    
    for ct in ct_names:
        data_path = raw_path + ct + '/raw/'
        if (ct == 'UPITT') or (ct == 'UTHSCSA') or (ct == 'UIOWA'):
            AKI_DX = pd.read_csv(data_path + "AKI_DX.csv", delimiter = ',', usecols=use_cols)
            #adjust the col order of UIOWA
            if ct == 'UIOWA':
                AKI_DX = AKI_DX[use_cols]
        elif (ct == 'UTSW'):
            AKI_DX = pd.read_csv(data_path + "AKI_DX.dsv", delimiter = '|', usecols=use_cols)
        elif (ct == 'MCW'):
            AKI_DX = pd.read_csv(data_path + "AKI_DX.dsv", delimiter = '|', usecols=list(map(str.lower, use_cols)))
            Upper_Case_Columns(AKI_DX)
        elif (ct == 'UMHC'):
            AKI_DX = pd.read_csv(data_path + "DEID_AKI_DX.csv", delimiter = ',', usecols=use_cols)
        elif (ct == 'UofU'):
            AKI_DX = pd.read_csv(data_path + "AKI_DX.csv", delimiter = '|', header=None, 
                                           skiprows = 1, usecols=[2, 6, 8, 9, 20])
            AKI_DX.columns = use_cols
        elif (ct == 'KUMC_ORCALE'):
            AKI_DX = pd.read_csv(data_path + "AKI_DX.csv", delimiter = ',', 
                                 usecols=['PATID', 'DX_DATE"+PD.DATE_SHIFT"', 'DX', 
                                          'DX_TYPE', 'DAYS_SINCE_ADMIT'])
            AKI_DX.columns = use_cols
        
        if ct not in ct_missing_DX_DATE:
            AKI_DX['DX_DATE'] = pd.to_datetime(AKI_DX['DX_DATE'], format = 'mixed')
            AKI_DX['DX_DATE'] = AKI_DX['DX_DATE'].dt.strftime('%Y-%m-%d')
            AKI_DX['DX_DATE'] = pd.to_datetime(AKI_DX['DX_DATE'], format = 'mixed')
            
        AKI_DX_dfs[ct] = AKI_DX
        
    return AKI_DX_dfs

In [ ]:
def process_query_by_AKI_DX(AKI_DX_dfs, AKI_query_dfs):
    ct_missing_DX_DATE = ['UTHSCSA', 'UTSW', 'UofU']
    
    for ct, AKI_DX in AKI_DX_dfs.items():
        print('\n' + ct + ':')
        
        #filter DX that is before the target hospitalization
        AKI_DX_to_filter = \
        AKI_query_dfs[ct][['PATID', 'ADMIT_DATE']].merge(AKI_DX, on = "PATID", how = "left")
        AKI_DX_to_filter.dropna(subset=['DX'], inplace = True)
        
        if ct in ct_missing_DX_DATE:
            AKI_DX_to_filter.loc[:, 'DX_DATE'] = \
            AKI_DX_to_filter.loc[:, 'ADMIT_DATE'] + \
            pd.to_timedelta(AKI_DX_to_filter.loc[:, 'DAYS_SINCE_ADMIT'], unit='D')
        
        AKI_DX_filtered = AKI_DX_to_filter[AKI_DX_to_filter.DX_DATE <= AKI_DX_to_filter.ADMIT_DATE]
        
        # Convert the mixed column to numeric, coercing errors to NaN
        AKI_DX['numeric_DX'] = pd.to_numeric(AKI_DX['DX'], errors='coerce')

        #Criteria 4 Exclude (6): Burn patients, Burns' ICD-9 codes are 940-949
        AKI_BURN = AKI_DX[(AKI_DX['numeric_DX'] >= 940) & (AKI_DX['numeric_DX'] <= 949) & (AKI_DX['DX_TYPE'] == 9)]
        #update query by filtering out patients with burns 
        AKI_query_dfs[ct] = AKI_query_dfs[ct][~AKI_query_dfs[ct].PATID.isin(AKI_BURN.PATID)]
        print('After Criteria 4 Exclude (6): burned patients, there are %s qualified patients left!' 
              %(len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())))
        records_num_df.loc['Burned patients excluded', ct] = \
        len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())

        #Criteria 4 Exclude (5): has pre-existing end stage renal disease, ICD-9 403, 404, 585
        ICD10_end_stage_renal = ['I12.0', 'I13.1', 'I13.2', 'N18.6', 'Z49.3']
        AKI_END_RENAL = AKI_DX[((((AKI_DX['numeric_DX'] >= 403) & (AKI_DX['numeric_DX'] < 405)) | 
                                 ((AKI_DX['numeric_DX'] >= 585) & (AKI_DX['numeric_DX'] < 586))) & (AKI_DX['DX_TYPE'] == 9)) | 
                               ((AKI_DX.DX.isin(ICD10_end_stage_renal)) & (AKI_DX['DX_TYPE'] == 10))]
        
        #Criteria 4 Exclude (3): has undergone any dialysis procedure or renal transplantation (RRT) prior to the visit
        DIA_RRT_DX_ICD10_DX = ['T86.10', 'T86.11', 'T86.12']
        AKI_DIA_RRT_DX = AKI_DX[(AKI_DX.DX.isin(DIA_RRT_DX_ICD10_DX)) & (AKI_DX.DX_TYPE == 10)]
        
        # Drop the auxiliary column 
        AKI_DX.drop('numeric_DX', axis=1, inplace=True)

        #update query by filtering out patients with end stage renal disease 
        AKI_query_dfs[ct] = AKI_query_dfs[ct][~AKI_query_dfs[ct].PATID.isin(AKI_END_RENAL.PATID)]
        AKI_query_dfs[ct] = AKI_query_dfs[ct][~AKI_query_dfs[ct].PATID.isin(AKI_DIA_RRT_DX.PATID)]
        print('After Criteria 4 Exclude (5): has pre-existing end stage renal disease, there are %s qualified patients left!' 
              %(len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())))
        records_num_df.loc['Patients with pre-existing end stage renal disease excluded', ct] = \
        len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())

In [ ]:
AKI_DX_dfs = read_AKI_DX(ct_names, raw_path)

In [ ]:
process_query_by_AKI_DX(AKI_DX_dfs, AKI_query_dfs)

# Preprocess Procedure Data and Join with Query

In [ ]:
#read patients' Procedure data
def read_AKI_PX(ct_names, raw_path):
    AKI_PX_dfs = dict()
    use_cols = ['PATID', 'PX', 'PX_TYPE']

    for ct in ct_names:
        data_path = raw_path + ct + '/raw/'
        if (ct == 'UPITT') or (ct == 'UTHSCSA') or (ct == 'UIOWA') or (ct == 'KUMC_ORCALE'):
            AKI_PX = pd.read_csv(data_path + "AKI_PX.csv", delimiter = ',', usecols = use_cols)
        elif (ct == 'UTSW'):
            AKI_PX = pd.read_csv(data_path + "AKI_PX.dsv", delimiter = '|', usecols = use_cols)
        elif (ct == 'MCW'):
            AKI_PX = pd.read_csv(data_path + "AKI_PX.dsv", delimiter = '|', usecols = list(map(str.lower, use_cols)))
            Upper_Case_Columns(AKI_PX)
        elif (ct == 'UMHC'):
            AKI_PX = pd.read_csv(data_path + "DEID_AKI_PX.csv", delimiter = ',', usecols = use_cols)
        elif (ct == 'UofU'):
            AKI_PX = pd.read_csv(data_path + "AKI_PX.csv", delimiter = '|', usecols = use_cols)
    
        AKI_PX_dfs[ct] = AKI_PX
        
    return AKI_PX_dfs

In [ ]:
def process_query_by_AKI_PX(AKI_PX_dfs, AKI_query_dfs):
    #Criteria 4 Exclude (3): has undergone any dialysis procedure or renal transplantation (RRT) prior to the visit 
    ICD10_DIA_RRT = ['5A1D00Z','5A1D60Z','5A1D70Z','5A1D80Z',
                     '5A1D90Z', '0TY00Z0','0TY00Z1',
                     '0TY00Z2','0TY10Z0','0TY10Z1',
                     '0TY10Z2', '0TB00ZZ','0TB10ZZ','0TT00ZZ',
                     '0TT10ZZ','0TT20ZZ']
    ICD9_DIA_RRT = ['39.93','39.95','54.98', '55.51',
                    '55.52','55.53','55.54','55.61','55.69']
    CPT_DIA_RRT = ['00868', '01990', '50300', '50320', 
                   '50323', '50325', '50327', '50328', 
                   '50329', '50340', '50360',
                    '50365', '50370', '50380', 
                   '90935', '90937']
    
    for ct, AKI_PX in AKI_PX_dfs.items():
        print('\n' + ct + ':')
        AKI_DIA_RRT = AKI_PX[((AKI_PX.PX.isin(ICD10_DIA_RRT)) & (AKI_PX.PX_TYPE == '10')) | 
                     ((AKI_PX.PX.isin(ICD9_DIA_RRT)) & (AKI_PX.PX_TYPE == '09')) |
                     ((AKI_PX.PX.isin(CPT_DIA_RRT)) & (AKI_PX.PX_TYPE == 'CH'))]
        AKI_query_dfs[ct] = AKI_query_dfs[ct][~AKI_query_dfs[ct].PATID.isin(AKI_DIA_RRT.PATID)]

        print('After Criteria 4 Exclude (3): has undergone any dialysis procedure or renal transplantation (RRT) prior to the visit, there are %d qualified patients left!' 
              %(len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())))
        records_num_df.loc['Patients with dialysis or renal transplantation excluded', ct] = \
        len(AKI_query_dfs[ct].ONSETS_ENCOUNTERID.unique())

In [ ]:
AKI_PX_dfs = read_AKI_PX(ct_names, raw_path)

In [ ]:
process_query_by_AKI_PX(AKI_PX_dfs, AKI_query_dfs)

# Preprocess SCr Data and Merge with Query

In [ ]:
#read Scr records, here we kept the historical records(DAYS_SINCE_ADMIT < 0)
def read_AKI_LAB_SCR(ct_names, raw_path):
    SCR_dfs = dict()
    use_cols = ['ONSETS_ENCOUNTERID','PATID','ENCOUNTERID','SPECIMEN_DATE','RESULT_NUM', 'DAYS_SINCE_ADMIT']

    for ct in ct_names:
        data_path = raw_path + ct + '/raw/'
        if (ct == 'UPITT') or (ct == 'UTHSCSA') or (ct == 'UIOWA'):
            SCR_df = pd.read_csv(data_path + "AKI_LAB_SCR.csv", delimiter = ',', usecols=use_cols)
        elif (ct == 'UTSW'):
            SCR_df = pd.read_csv(data_path + "AKI_LAB_SCR.dsv", delimiter = '|', usecols=use_cols)
        elif (ct == 'MCW'):
            SCR_df = pd.read_csv(data_path + "AKI_LAB_SCR.dsv", delimiter = '|', usecols=list(map(str.lower, use_cols)))
            Upper_Case_Columns(SCR_df)
        elif (ct == 'UMHC'):
            SCR_df = pd.read_csv(data_path + "DEID_AKI_LAB_SCR.csv", delimiter = ',', usecols=use_cols)
        elif (ct == 'UofU'):
            SCR_df = pd.read_csv(data_path + "AKI_LAB_SCR.csv", delimiter = '|', usecols=use_cols)
        elif (ct == 'KUMC_ORCALE'):
            use_cols = ['ONSETS_ENCOUNTERID','PATID','ENCOUNTERID',
                        'SPECIMEN_DATE"+PD.DATE_SHIFT"','RESULT_NUM', 'DAYS_SINCE_ADMIT']
            SCR_df = pd.read_csv(data_path + "AKI_LAB_SCR.csv", delimiter = ',', usecols=use_cols)
            SCR_df.columns = ['ONSETS_ENCOUNTERID','PATID','ENCOUNTERID', 'SPECIMEN_DATE','RESULT_NUM', 
                              'DAYS_SINCE_ADMIT']

        SCR_dfs[ct] = SCR_df
        
    return SCR_dfs

In [ ]:
def filter_groups(group):
    return group['RESULT_NUM'].max() <= max_SCR

In [ ]:
def merge_SCR_query_and_process(SCR_dfs, AKI_query_dfs, mode = 'BEFORE_ONSET'):
    AKI_SCR_dfs = dict()
    
    for ct, Scr in SCR_dfs.items():
        print('\n' + ct + ':')
        #filter out Non-AKI patients
        AKI_Scr = AKI_query_dfs[ct].merge(Scr, how = 'inner', on = ['PATID', 'ONSETS_ENCOUNTERID'])
        
        # Converting string data type into datetime object
        AKI_Scr['SPECIMEN_DATE'] = pd.to_datetime(AKI_Scr['SPECIMEN_DATE'], format='mixed')
        # Extract just the date part(Only needed by UMHC data)
        if ct == 'UMHC':
            AKI_Scr['SPECIMEN_DATE'] = AKI_Scr['SPECIMEN_DATE'].dt.date
            AKI_Scr['SPECIMEN_DATE'] = pd.to_datetime(AKI_Scr['SPECIMEN_DATE'])
            
        #some DAYS_SINCE_ADMIT need to be corrected
        AKI_Scr['DAYS_SINCE_ADMIT'] = (AKI_Scr['SPECIMEN_DATE'] - AKI_Scr['ADMIT_DATE']).dt.days
        
        #sort the df by ONSETS_ENCOUNTERID
        #Tip: 'ONSETS_ENCOUNTERID'represents the ID for current hospitalization， 'ENCOUNTERID' is bound to the test day，can be in the past，
        #if it happend in the past, then DAYS_SINCE_ADMIT is negative, means that it happended before the date of admit date represented by
        #'ONSETS_ENCOUNTERID', if it happens now, then 'ENCOUNTERID' should be same as 'ONSETS_ENCOUNTERID'.
        #Here, ADMIT_DATE, DISCHARGE_DATE, AKI123_ONSET, EARLIEST_ONSET is tied to ONSETS_ENCOUNTERID, representing current hospitalization,
        #while, SPECIMEN_DATE is tied to ENCOUNTERID, and DAYS_SINCE_ADMIT is relative to ONSETS_ENCOUNTERID/ADMIT_DATE
        AKI_Scr = AKI_Scr.sort_values(by=['ONSETS_ENCOUNTERID', 'SPECIMEN_DATE'])

        #Multiple measurements of SCr one the same day are averaged
        AKI_Scr['RESULT_NUM'] = AKI_Scr.groupby(['PATID', 'SPECIMEN_DATE'])['RESULT_NUM'].transform(np.mean)
        AKI_Scr.drop_duplicates(subset=['PATID', 'SPECIMEN_DATE'], inplace=True)

        print('After Averaging records on the same day, there are %s qualified records left!' %len(AKI_Scr.ONSETS_ENCOUNTERID.unique()))
        
        
        AKI_Scr.drop_duplicates(subset=['PATID', 'SPECIMEN_DATE'], inplace=True)
        
        #((at least n SCr records)---|Admit|---(at least n' SCr records)---|AKI Onset|---(no requirement)---|Discharge|)
        #Criteria 3: at least n SCr records between Admit date and AKI onset date, 
        #and also at least n' SCr history records within 30 days before the ONSET day,
        #total time series interval is 30 days before the onset day
        AKI_Scr['DAYS_BEFORE_ONSET'] = (AKI_Scr['SPECIMEN_DATE'] - AKI_Scr['EARLIEST_ONSET']).dt.days
        
        if mode == 'BEFORE_ONSET':
            #drop records after onset day(we should include onset dat itself), or historical records not within 7 day time interval
            AKI_Scr = AKI_Scr[(AKI_Scr.DAYS_BEFORE_ONSET <= 0) & (AKI_Scr.DAYS_BEFORE_ONSET > -time_interval_before_ONSET)]

            #sub-requirement 2: admit-onset period # of record should be at least 'n_rec_after_admit'
            scr_before_onset = AKI_Scr[(AKI_Scr.DAYS_BEFORE_ONSET < 0) & ((AKI_Scr.DAYS_SINCE_ADMIT >=0))].groupby('ONSETS_ENCOUNTERID').count()
            encounters_to_keep2 = scr_before_onset[scr_before_onset.SPECIMEN_DATE >= n_rec_after_admit].index
            AKI_Scr = AKI_Scr[AKI_Scr.ONSETS_ENCOUNTERID.isin(encounters_to_keep2)]

            print('After Criteria 3 sub-req 2: admit-onset period # of record should be at least %s, there are %s qualified records left!' 
                  %(n_rec_after_admit, len(AKI_Scr.ONSETS_ENCOUNTERID.unique())))

            #sub-requirement 3: at least 1 record on the AKI onset day
            scr_on_onset = AKI_Scr[AKI_Scr.DAYS_BEFORE_ONSET == 0].groupby('ONSETS_ENCOUNTERID').count()
            encounters_to_keep3 = scr_on_onset[scr_on_onset.SPECIMEN_DATE >= 1].index
            AKI_Scr = AKI_Scr[AKI_Scr.ONSETS_ENCOUNTERID.isin(encounters_to_keep3)]

            print('After Criteria 3 sub-req 3: at least 1 record on the AKI onset day, there are %s qualified records left!' 
                  %len(AKI_Scr.ONSETS_ENCOUNTERID.unique()))

            #sub-requirement 4: total record count should not below the criteria
            scr_rec_total = AKI_Scr.groupby('ONSETS_ENCOUNTERID').count()
            n_records_req = int((1 - missing_percent_allowed) * time_interval_before_ONSET)
            encounters_to_keep4 = scr_rec_total[scr_rec_total.SPECIMEN_DATE >= n_records_req].index
            AKI_Scr = AKI_Scr[AKI_Scr.ONSETS_ENCOUNTERID.isin(encounters_to_keep4)]
            print('After Criteria 3 sub-req 4: total record count should not below the criteria - %s, there are %s qualified records left!' 
                  %(n_records_req, len(AKI_Scr.ONSETS_ENCOUNTERID.unique())))
            
            #sub-requirement 5: max SCr should not excceed a certain number
            AKI_Scr = AKI_Scr.groupby('ONSETS_ENCOUNTERID').filter(filter_groups)
            print('After Criteria 3 sub-req 5: should not exceed max SCr, there are %s qualified records left!' 
                  %len(AKI_Scr.ONSETS_ENCOUNTERID.unique()))
            records_num_df.loc['Patient not satisfying SCr trajectory requirements excluded', ct] = \
            len(AKI_Scr.ONSETS_ENCOUNTERID.unique())
        
        elif mode == 'AFTER_ONSET':
            #here we only require time after onset but not limit window because we also want to analyse time window prior to discharge
            AKI_Scr = AKI_Scr[AKI_Scr.DAYS_BEFORE_ONSET > 0]
        elif mode == 'BEFORE_ADMIT':
            #here we apply time window since we will only plot trajectory
            AKI_Scr = AKI_Scr[(AKI_Scr.SPECIMEN_DATE < AKI_Scr.ADMIT_DATE) & \
                              (AKI_Scr.SPECIMEN_DATE >= AKI_Scr.ADMIT_DATE - pd.Timedelta(days=7))]
        
        AKI_SCR_dfs[ct] = AKI_Scr
    
    return AKI_SCR_dfs

In [ ]:
SCR_dfs = read_AKI_LAB_SCR(ct_names, raw_path)

In [ ]:
AKI_SCR_dfs_before_onset = merge_SCR_query_and_process(SCR_dfs, AKI_query_dfs, mode = 'BEFORE_ONSET')

In [ ]:
AKI_SCR_dfs_after_onset = merge_SCR_query_and_process(SCR_dfs, AKI_query_dfs, mode = 'AFTER_ONSET')

In [ ]:
AKI_SCR_dfs_before_admit = merge_SCR_query_and_process(SCR_dfs, AKI_query_dfs, mode = 'BEFORE_ADMIT')

# Merge All Data and Output to Directory

In [ ]:
# Initialize an empty DataFrame to hold the concatenated result
concat_SCR_df_before_onset = pd.DataFrame()

# Loop through the dict of dataframes and concatenate
for ct, AKI_Scr_before_onset in AKI_SCR_dfs_before_onset.items():
    #previously there was no center name col, here we add it
    if ct == 'KUMC_ORCALE':
        AKI_Scr_before_onset['CENTER_NAME'] = 'KUMC'
    else:
        AKI_Scr_before_onset['CENTER_NAME'] = ct
    concat_SCR_df_before_onset = pd.concat([concat_SCR_df_before_onset, 
                                            AKI_Scr_before_onset], ignore_index=True)

In [ ]:
concat_SCR_df_before_onset['ONSETS_ENCOUNTERID'] = concat_SCR_df_before_onset['ONSETS_ENCOUNTERID'].astype(str)

In [ ]:
#here we drop these records becasue they are the only 2 patients that have the same onset_encounterID
#And it will bring error in the following pipelines
concat_SCR_df_before_onset['unique_col2_count'] = concat_SCR_df_before_onset.groupby('ONSETS_ENCOUNTERID')['CENTER_NAME'].transform('nunique')
dup_result = concat_SCR_df_before_onset[concat_SCR_df_before_onset['unique_col2_count'] > 1]
dup_result = list(dup_result.ONSETS_ENCOUNTERID.unique())
concat_SCR_df_before_onset = \
concat_SCR_df_before_onset[~concat_SCR_df_before_onset.ONSETS_ENCOUNTERID.isin(dup_result)]
concat_SCR_df_before_onset.drop('unique_col2_count', axis = 1, inplace = True)

In [ ]:
dup_result

In [ ]:
len(concat_SCR_df_before_onset.ONSETS_ENCOUNTERID.unique())

In [ ]:
for ct_name in ct_names:
    if ct_name == 'KUMC_ORCALE':
        ct_name = 'KUMC'
    unique_encounter_n = len(concat_SCR_df_before_onset[concat_SCR_df_before_onset.CENTER_NAME == ct_name].ONSETS_ENCOUNTERID.unique())
    print(ct_name, unique_encounter_n)
    print()

In [ ]:
# Initialize an empty DataFrame to hold the concatenated result
concat_SCR_df_after_onset = pd.DataFrame()

# Loop through the dict of dataframes and concatenate
for ct, AKI_Scr_after_onset in AKI_SCR_dfs_after_onset.items():
    #previously there was no center name col, here we add it
    if ct == 'KUMC_ORCALE':
        AKI_Scr_after_onset['CENTER_NAME'] = 'KUMC'
    else:
        AKI_Scr_after_onset['CENTER_NAME'] = ct
    concat_SCR_df_after_onset = pd.concat([concat_SCR_df_after_onset, 
                                           AKI_Scr_after_onset], ignore_index=True)

In [ ]:
concat_SCR_df_after_onset['ONSETS_ENCOUNTERID'] = concat_SCR_df_after_onset['ONSETS_ENCOUNTERID'].astype(str)

In [ ]:
#here we filter out records that is not in before onset collection
concat_SCR_df_after_onset_filtered = \
concat_SCR_df_after_onset[concat_SCR_df_after_onset.ONSETS_ENCOUNTERID.isin(concat_SCR_df_before_onset.ONSETS_ENCOUNTERID)]

In [ ]:
len(concat_SCR_df_after_onset_filtered.ONSETS_ENCOUNTERID.unique())

In [ ]:
# Initialize an empty DataFrame to hold the concatenated result
concat_SCR_df_before_admit = pd.DataFrame()

# Loop through the dict of dataframes and concatenate
for ct, AKI_Scr_before_admit in AKI_SCR_dfs_before_admit.items():
    #previously there was no center name col, here we add it
    if ct == 'KUMC_ORCALE':
        AKI_Scr_before_admit['CENTER_NAME'] = 'KUMC'
    else:
        AKI_Scr_before_admit['CENTER_NAME'] = ct
    concat_SCR_df_before_admit = pd.concat([concat_SCR_df_before_admit, 
                                           AKI_Scr_before_admit], ignore_index=True)

In [ ]:
concat_SCR_df_before_admit['ONSETS_ENCOUNTERID'] = concat_SCR_df_before_admit['ONSETS_ENCOUNTERID'].astype(str)

In [ ]:
#here we filter out records that is not in before onset collection
concat_SCR_df_before_admit_filtered = \
concat_SCR_df_before_admit[concat_SCR_df_before_admit.ONSETS_ENCOUNTERID.isin(concat_SCR_df_before_onset.ONSETS_ENCOUNTERID)]

# Save DF

In [ ]:
concat_SCR_df_before_onset.to_csv('/blue/yonghui.wu/lideyi/AKI_subphenotyping_project_v3/full_data/concat_SCR_data_before_onset.csv', index=False)

In [ ]:
concat_SCR_df_after_onset_filtered.to_csv('/blue/yonghui.wu/lideyi/AKI_subphenotyping_project_v3/full_data/concat_SCR_data_after_onset.csv', index=False)

In [ ]:
concat_SCR_df_before_admit_filtered.to_csv('/blue/yonghui.wu/lideyi/AKI_subphenotyping_project_v3/full_data/concat_SCR_data_before_admit.csv', index=False)

In [ ]:
records_num_df['Total'] = records_num_df.sum(axis = 1)

In [ ]:
def add_percentage(col):
    total = col.iloc[0]  # first row (cluster total) is the total count
    return col.map(lambda entry: f"{entry}({(entry / total) * 100:.2f}%)" if total != 0 else "0(0.00%)")

In [ ]:
records_num_df_penct = records_num_df.apply(add_percentage, axis = 0)

In [ ]:
records_num_df_penct

In [ ]:
records_num_df_penct.to_csv('./Tables/Patient_Screening.csv', index = True)